# Structure of Spotify Playlist Data
1. Big dict with key: "playlists"
2. list of all playlists
3. each element of this list is a dictionary with these important keys: "name" and "items"
4. "items" is a list of dictionaries, each represening one song
5. "track" is the only important key in this dictionary
6. "track"'s value is another dictionary with these keys "trackName" "artistName" "albumName"

In [1]:
#Always run me
import json
from collections import defaultdict
from bs4 import BeautifulSoup
import urllib.request
import re
import nltk.corpus
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/emerald/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
with open("/Users/emerald/Desktop/SpotifyData/Playlist1.json") as json_file:
    playlist_data = json.load(json_file)

with open("/Users/emerald/Desktop/SpotifyData/stop_words") as file:
    stop_words = file.read()
stop_words = stop_words.strip()
stop_words = stop_words.split()
stop_words.append("chorus")
stop_words.append("verse")
stop_words.append("prechorus")
stop_words.append("bridge")

In [3]:
#let's make a dictionary

playlist_dict = {}
for b in playlist_data["playlists"]: #creates a dictionary of playlists with playlists titles as keys and a list with each song represented as the dict from #6 as the values
    playlist_dict[b["name"]] = []
    for i in b["items"]:
        playlist_dict[b["name"]].append(i["track"])
        
to_remove= ["Liked from Radio" , "Instrumental Study" , "White Noise for Mom" , "RENT" , "StarKid" , "Disney", "Carols that Clang", "Demi-Groove 2018", "Ocean Waves Relax", "yeah I listen to indie rock", "Guardians of the Galaxy Soundtracks", "Jack's Shaky Knees Picks", "12/10/16 (Finals)", "#makeobamapresidentagain", "fucking superb you funky little dryad", "9/9/16", "Ceramics", "fake obscure music", "Let's Mix It Uppp","i am my parents' daughter", "2/19/17", "Nostalgia", "me, an intellectual", "Taylor Swift Deep-Cuts", "gotta chill w/ some new tunes", "wail with me, Bog Man", "Florence Goes Off", "we're staying in the south boyees", "Not All Country is Bad", "please don't @ me"]
for b in to_remove: #gets rid of unneccesary playlists
    del playlist_dict[b]


In [4]:
#make a global vocab of songs
global_songs = []
artists = []
for b in playlist_dict: #makes a global vocab of all songs as [artist, song] len 554
    try:
        for i in playlist_dict[b]:
            artist = (i["artistName"])
            artist = artist.lower()
            artist = re.sub(r'[^\w]',"",artist)
            song = (i["trackName"])
            song = song.lower()
            song = re.sub(r'[^\w]',"",song)
            artists.append(artist)
            if [artist, song] not in global_songs:
                global_songs.append([artist, song])
                
    except:
        "TypeError"    


In [5]:
with open("/Users/emerald/Desktop/SpotifyData/caseys_songs.json") as json_file:
    casey_songs = json.load(json_file)
with open("/Users/emerald/Desktop/SpotifyData/caseys_songs_2.json") as json_file:
    casey_2 = json.load(json_file)
with open("/Users/emerald/Desktop/SpotifyData/caseys_songs_3.json") as json_file:
    casey_3 = json.load(json_file)

remove = ["theamazingdevil", "variousartists", "originalbroadwaycastofonceanewmusical", "bearmccreary", "highschoolmusicalcast", "benplatt", "barrettwilbertweed", "chasinglovely"]   
trimmed_global_songs = []
for i in global_songs:
    if i[1] not in list(casey_songs.keys()) and list(casey_2.keys()) and list(casey_3.keys()) and "outlandertheskyeboatsongcastleleochversionfeatrayayarbroug": 
        if i[0] not in remove:
            trimmed_global_songs.append(i)

for i in trimmed_global_songs:
    print(i)
print(len(trimmed_global_songs))

['hozier', 'angelofsmalldeathandthecodeinescene']
['hozier', 'inaweekfeatkarencowley']
['hozier', 'cherrywinelive']
['hozier', 'ninacriedpowerfeatmavisstaples']
['hozier', 'betterlovefromthelegendoftarzansingleversion']
['hozier', 'worksongliveinamericaspringsummer2015']
['theheadandtheheart', 'runningthroughhell']
['falloutboy', 'disloyalorderofwaterbuffaloes']
['noahcyrus', 'lonely']
['kesha', 'praying']
['vampireweekend', 'thislife']
['harrystyles', 'watermelonsugar']
['zacbrownband', 'highway20ride']
['zacbrownband', 'whateveritis']
['vancejoy', 'layitonme']
['vancejoy', 'georgia']
['vancejoy', 'messismine']
['georgeezra', 'budapest']
['theheadandtheheart', 'riversandroads']
['therollingstones', 'wildhorses']
['thebeatles', 'herecomesthesunremastered2009']
['theband', 'theweightremastered']
['theband', 'atlanticcity']
['theallmanbrothersband', 'midnightrider']
['beyoncé', 'xo']
['jasonisbellandthe400unit', 'ifwewerevampires']
['themarshalltuckerband', 'cantyousee']
['thedecemberist

The next bit of code created bags of words for each song. It takes forever to run so I saved everything to "/Users/emerald/Desktop/SpotifyData/caseys_songs.json" for later use. The dictionary it creates is called bags_of_words and each key is a song name and each value is the bag of words for that song

In [ ]:
#scrape lyrics to make the bag of words
bags_of_words = {} #dictionary where key is song name value is bag of words for that song

for s in trimmed_global_songs:
    bag = defaultdict(int)
    try:
        webpage = "https://www.azlyrics.com/lyrics/"+s[0]+"/"+s[1]+".html"
        websource = urllib.request.urlopen(webpage)
        soup = BeautifulSoup(websource.read(), "html.parser")
        lyrics = soup.find("div", {"class": "col-xs-12 col-lg-8 text-center"}).find("div", {"class": None}).get_text() #pulls lyrics out of soup as a big string
        lyrics = lyrics.lower()
        lyrics = lyrics.strip()
        lyrics = lyrics.split()
        for i in lyrics:
            if i not in stop_words:
                t = re.sub(r"\W$","",i)
                bag[t]+=1
        bags_of_words[s[1]] = bag
        print(webpage+ " worked")
    except:
        "HTTPError"
        print(webpage+ " failed")

https://www.azlyrics.com/lyrics/hozier/angelofsmalldeathandthecodeinescene.html failed
https://www.azlyrics.com/lyrics/hozier/inaweekfeatkarencowley.html failed
https://www.azlyrics.com/lyrics/hozier/cherrywinelive.html failed
https://www.azlyrics.com/lyrics/hozier/ninacriedpowerfeatmavisstaples.html failed
https://www.azlyrics.com/lyrics/hozier/betterlovefromthelegendoftarzansingleversion.html failed
https://www.azlyrics.com/lyrics/hozier/worksongliveinamericaspringsummer2015.html failed
https://www.azlyrics.com/lyrics/theheadandtheheart/runningthroughhell.html failed
https://www.azlyrics.com/lyrics/falloutboy/disloyalorderofwaterbuffaloes.html failed


In [ ]:
#writing scraped data to an outfile so we don't have to do this again
with open("/Users/emerald/Desktop/SpotifyData/caseys_songs_4.json", "w") as f:
    json.dump(bags_of_words, f,  ensure_ascii=False)

In [ ]:
#counter = 0
#for i in global_songs:
 #   print(i, counter)
  #  counter+=1

In [ ]:
print(bags_of_words)